In [1]:
import pandas as pd

In [27]:
df= pd.read_csv("/home/jovyan/SnowEx-Map-Tool/GM_Pits.csv")

In [28]:
df.columns

Index(['Location', 'Site', 'PitID', 'Date/Local Standard Time', 'UTM Zone',
       'Easting (m)', 'Northing (m)', 'Latitude (deg)', 'Longitude (deg)',
       'Precip Type', 'Precip Rate', 'Sky', 'Wind', 'Ground Condition',
       'Ground Roughness', 'Ground Vegetation',
       'Height of Ground Vegetation (cm)', 'Canopy'],
      dtype='object')

In [11]:
import folium
from folium.plugins import MarkerCluster

# Create a map centered around the mean location with ESRI Imagery
m = folium.Map(location=[df['Latitude (deg)'].mean(), df['Longitude (deg)'].mean()],
               zoom_start=12, 
               tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}', 
               attr='Esri')

# Add an additional tile layer for labels (state names, city names, roads, etc.)
folium.TileLayer(
    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/Reference/World_Boundaries_and_Places/MapServer/tile/{z}/{y}/{x}',
    attr='Esri',
    name='Labels',
    overlay=True,
    control=True
).add_to(m)

# Add a marker cluster for better visualization
marker_cluster = MarkerCluster().add_to(m)

# Add the points to the map
for _, row in df.iterrows():
    folium.Marker(
        location=[row['Latitude (deg)'], row['Longitude (deg)']],
        popup=f"PitID: {row['PitID']}<br>Site: {row['Site']}<br>Date: {row['Date/Local Standard Time']}",
    ).add_to(marker_cluster)

# Save the map to an HTML file
m.save('/home/jovyan/SnowEx-Map-Tool/map.html')



In [12]:
m

In [23]:
from datetime import datetime
from snowexsql.api import PointMeasurements

# Function to retrieve point measurements for a specific pit within the specified date range
def get_point_measurements_for_pit(pit_id):
    measurements = PointMeasurements.from_filter(
        site_name="Grand Mesa",
        date_less_equal=datetime(2020, 5, 31),
        date_greater_equal=datetime(2019, 12, 1),
        limit=10
    )
    # Filter the measurements by PitID if necessary
    # Ensure measurements is a list of dicts or objects with attributes, adapt if it's different
    filtered_measurements = [m for m in measurements if m['pit_id'] == pit_id]
    return filtered_measurements


In [25]:
from datetime import datetime
from snowexsql.api import PointMeasurements

# Function to retrieve point measurements and print their format
def print_measurements_format():
    measurements = PointMeasurements.from_filter(
        site_name="Grand Mesa",
        date_less_equal=datetime(2020, 5, 31),
        date_greater_equal=datetime(2019, 12, 1),
        limit=10
    )
    print(measurements)

# Call the function to inspect data format
print_measurements_format()


  version_number equipment       value   latitude   longitude      northing  \
0           None      None  282.644548  39.017927 -108.167277  4.322584e+06   
1           None      None  282.644548  39.017926 -108.167276  4.322584e+06   
2           None      None  282.644548  39.017926 -108.167275  4.322584e+06   
3           None      None  282.644548  39.017926 -108.167274  4.322584e+06   
4           None      None  282.644548  39.017925 -108.167274  4.322584e+06   
5           None      None  282.644548  39.017925 -108.167273  4.322584e+06   
6           None      None  282.644548  39.017925 -108.167272  4.322584e+06   
7           None      None  282.644548  39.017924 -108.167271  4.322584e+06   
8           None      None  282.644548  39.017924 -108.167270  4.322584e+06   
9           None      None  282.644548  39.017924 -108.167269  4.322584e+06   

         easting elevation  utm_zone                            geom  ...  \
0  745249.475031      None        12  POINT (745249.4

In [31]:
import folium
from folium.plugins import MarkerCluster
import pandas as pd
# Create a map centered around the mean location with ESRI Imagery
m = folium.Map(location=[df['Latitude (deg)'].mean(), df['Longitude (deg)'].mean()],
               zoom_start=12, 
               tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}', 
               attr='Esri')

# Add an additional tile layer for labels (state names, city names, roads, etc.)
folium.TileLayer(
    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/Reference/World_Boundaries_and_Places/MapServer/tile/{z}/{y}/{x}',
    attr='Esri',
    name='Labels',
    overlay=True,
    control=True
).add_to(m)

# Add a marker cluster for better visualization
marker_cluster = MarkerCluster().add_to(m)

# Function to format measurement info for popup
def get_measurement_info(pit_id):
    measurements = df[df['PitID'] == pit_id]
    if not measurements.empty:
        info = "<br>".join([
            f"Date: {row['Date/Local Standard Time']}" 
            for _, row in measurements.iterrows()
        ])
    else:
        info = "No measurements available"
    return info

# Add the points to the map with measurement info in popups
for _, row in df.iterrows():
    pit_id = row['PitID']
    measurement_info = get_measurement_info(pit_id)
    
    folium.Marker(
        location=[row['Latitude (deg)'], row['Longitude (deg)']],
        popup=f"PitID: {pit_id}<br>Site: {row['Site']}<br>{measurement_info}",
    ).add_to(marker_cluster)

# Save the map to an HTML file
m.save('/home/jovyan/SnowEx-Map-Tool/map.html')


In [32]:
m